<a href="https://colab.research.google.com/github/KevinTheRainmaker/Recommendation_Algorithms/blob/main/colab/fastcampus/Recommender_Trial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 간단한 추천시스템 만들기

- Movielens Dataset 이용
- 3가지 방식의 간단한 추천 시스템을 구현해보고 평가
  - 영화 평균 평점기반 예측
  - 사용자 평균 평점기반 예측
  - Rule-Based 평점 예측
  
- 평가 기준은 RMSE

## 0. Configuration


In [1]:
import os
from tqdm import tqdm # 진행률 Process Bar
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from scipy.sparse import csr_matrix # 압축희소행렬(Compressed Sparse Row)

import warnings
warnings.filterwarnings('ignore')

 **CSR이란**

 Compressed sparse row (CSR): 가로의 순서대로 재정렬하는 방법으로 행에 관여하여 정리 압축

장점: 산술연산, 행 슬라이싱, 행렬 벡터 곱이 효율적이고 빠름

단점: 느린 열 슬라이싱(Compressed Sparse Column 고려), 희소성 구조 변화 시 연산비용 큼 (LIL, DOK 고려)

## 1. Dataset

- ratings.csv
- movies.csv
- tags.csv

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
path = '/content/drive/My Drive/data/ml-latest-small'

In [5]:
ratings_df = pd.read_csv(os.path.join(path, 'ratings.csv'), encoding='utf-8')
movies_df = pd.read_csv(os.path.join(path, 'movies.csv'), index_col='movieId', encoding='utf-8')
tags_df = pd.read_csv(os.path.join(path, 'tags.csv'), encoding='utf-8')

In [6]:
print(ratings_df.shape)
print(ratings_df.sample(5))

(100836, 4)
       userId  movieId  rating   timestamp
67883     438     4052     3.5  1105911093
49707     318    92637     3.5  1426784369
35782     241   118700     3.0  1447536831
96741     603     2763     2.0   953926333
53002     349      434     3.0   834750699


In [7]:
print(movies_df.shape)
print(movies_df.sample(5))

(9742, 2)
                                                     title                                genres
movieId                                                                                         
122886   Star Wars: Episode VII - The Force Awakens (2015)  Action|Adventure|Fantasy|Sci-Fi|IMAX
1023           Winnie the Pooh and the Blustery Day (1968)            Animation|Children|Musical
841      Eyes Without a Face (Yeux sans visage, Les) (1...                                Horror
6702              Dickie Roberts: Former Child Star (2003)                                Comedy
5450                               Lovely & Amazing (2001)                  Comedy|Drama|Romance


In [8]:
print(tags_df.shape)
print(tags_df.sample(5))

(3683, 4)
      userId  movieId          tag   timestamp
3454     599      296         gore  1498456511
3055     567     6818   disturbing  1525282337
2712     477    68358        space  1262795932
2592     477     4454  imagination  1245030178
3274     567   152077       creepy  1525287447


## 2. About ratings_df

In [9]:
num_users = ratings_df['userId'].unique()
num_movies = ratings_df['movieId'].unique()

print('Total Users:', len(num_users))
print('Total Movies:', len(num_movies))

Total Users: 610
Total Movies: 9724


In [10]:
# pivot ratings into movie features

user_movie_matrix = ratings_df.pivot(
    index = 'movieId',
    columns = 'userId',
    values = 'rating'
).fillna(0) # NaN to 0

# convert dataframe of movie features to scipy sparse matrix
sparse_mat = csr_matrix(user_movie_matrix.values)

In [11]:
print(user_movie_matrix)

userId   1    2    3    4    5    6    7    ...  604  605  606  607  608  609  610
movieId                                     ...                                   
1        4.0  0.0  0.0  0.0  4.0  0.0  4.5  ...  3.0  4.0  2.5  4.0  2.5  3.0  5.0
2        0.0  0.0  0.0  0.0  0.0  4.0  0.0  ...  5.0  3.5  0.0  0.0  2.0  0.0  0.0
3        4.0  0.0  0.0  0.0  0.0  5.0  0.0  ...  0.0  0.0  0.0  0.0  2.0  0.0  0.0
4        0.0  0.0  0.0  0.0  0.0  3.0  0.0  ...  0.0  0.0  0.0  0.0  0.0  0.0  0.0
5        0.0  0.0  0.0  0.0  0.0  5.0  0.0  ...  3.0  0.0  0.0  0.0  0.0  0.0  0.0
...      ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...
193581   0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0  0.0  0.0  0.0  0.0
193583   0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0  0.0  0.0  0.0  0.0
193585   0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0  0.0  0.0  0.0  0.0
193587   0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0  0.0  0.0  0.0  0.0
1936

In [12]:
print(sparse_mat)

  (0, 0)	4.0
  (0, 4)	4.0
  (0, 6)	4.5
  (0, 14)	2.5
  (0, 16)	4.5
  (0, 17)	3.5
  (0, 18)	4.0
  (0, 20)	3.5
  (0, 26)	3.0
  (0, 30)	5.0
  (0, 31)	3.0
  (0, 32)	3.0
  (0, 39)	5.0
  (0, 42)	5.0
  (0, 43)	3.0
  (0, 44)	4.0
  (0, 45)	5.0
  (0, 49)	3.0
  (0, 53)	3.0
  (0, 56)	5.0
  (0, 62)	5.0
  (0, 63)	4.0
  (0, 65)	4.0
  (0, 67)	2.5
  (0, 70)	5.0
  :	:
  (9700, 337)	2.5
  (9701, 337)	3.0
  (9702, 183)	4.0
  (9702, 247)	3.5
  (9703, 317)	2.5
  (9704, 209)	1.0
  (9705, 461)	2.5
  (9706, 49)	3.5
  (9707, 337)	1.5
  (9708, 337)	4.0
  (9709, 337)	1.0
  (9710, 337)	1.5
  (9711, 337)	1.0
  (9712, 337)	1.0
  (9713, 183)	4.5
  (9714, 183)	3.5
  (9715, 183)	3.0
  (9716, 183)	4.0
  (9717, 183)	4.0
  (9718, 183)	3.5
  (9719, 183)	4.0
  (9720, 183)	3.5
  (9721, 183)	3.5
  (9722, 183)	3.5
  (9723, 330)	4.0


In [14]:
print(sparse_mat.indptr)
print(sparse_mat.indices)
print(sparse_mat.data)

[     0    215    325 ... 100834 100835 100836]
[  0   4   6 ... 183 183 330]
[4.  4.  4.5 ... 3.5 3.5 4. ]


In [15]:
user_info_df = pd.DataFrame(data = [sum(list(user_movie_matrix[int(x)].value_counts())[1:]) for x in user_movie_matrix.columns],
                            index = user_movie_matrix.columns, 
                            columns = ['movies_rated'])

# sum(list(user_movie_matrix[1].value_counts())[1:])

In [16]:
user_info_df

,movies_rated
userId,
1,232
2,29
3,39
4,216
5,44
...,...
606,1115
607,187
608,831


In [18]:
movie_info_df = pd.DataFrame(data = [sum(list(user_movie_matrix.loc[int(x)].value_counts())[1:]) for x in user_movie_matrix.index],
                            index = user_movie_matrix.index, 
                            columns = ['movies_rated'])

In [19]:
movie_info_df

,movies_rated
movieId,
1,215
2,110
3,52
4,7
5,49
...,...
193581,1
193583,1
193585,1


## 3. Split Dataset